# Import

In [ ]:
# import base pacakges
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # 3D plots
import matplotlib.tri as tri # make surface plot
from pathlib import Path # to extract data from registry
import re # to extract number in name
import scipy.optimize as opt # data fitting
import pandas as pd # data frames
from mpl_toolkits.axes_grid1 import make_axes_locatable #adjust colorbars to axis
from tqdm.notebook import tqdm # loading bar
import ipywidgets as widgets #widgets

In [ ]:
# import custom made pacakges
# directory navigation
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
parentparentdir = os.path.dirname(parentdir)
sys.path.insert(0,parentparentdir) 
# import 
from modules.config import c,d_in,foc,freq,λ,ω_0_in_list,ω_0_in # standard parameters
from modules.beam_waist_functions import cal_ω_0_out_and_d_out, create_plot # theoretical beam waist functions
from modules.heterodyne_functions import I_comb

In [ ]:
# # # # # # # Turn off future warnings # # # # # 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### Custom parameters

In [ ]:
# # Define Physical constants:
# c=2.99792458E11 # in mm/s

# # Define Input Paramters:
# d_in=449.7# in mm
# freq=95E9
# # mirror param:
# foc=339.5
# λ=c/freq
# # manufacturers paramas:
# ω_0_in_list={70:7.85,75:7.67,80:7.47,85:7.26,90:7.03,95:7.16,100:7.23,105:7.26,110:7.24} #GHz:mm
# ω_0_in=ω_0_in_list[min(ω_0_in_list.keys(), key=lambda x:abs(x-freq/1E9))]

# Import data from directory

In [ ]:
# path of measurements
p=Path('./Measurements/serious measurements')
# list all main measurement folders
print("Choose folder:")
dir_names=list([x.name for x in p.iterdir() if x.is_dir() and x.name.startswith("Distance")])
menu=widgets.Dropdown(
    options={dir_names[i]:i for i in range(len(dir_names))},
    value=2,
    description='Folder:')
display(menu)

In [ ]:
paths=list([x for x in p.iterdir() if x.is_dir() and x.name.startswith("Distance")][menu.value].glob('./*.dat')) #use ** to also include subregistries
#remember to change name if directory name is changed
data=[]
for path in paths:
    data.append(np.genfromtxt(path,skip_header=1)[:,:3]) #all rows, only the first 3 columns
# print current directory
print("current directory:\n{}".format(dir_names[menu.value]))
#extract distance from directory name
pattern=re.compile(r"(\D+)(\d+)")
dir_dist=np.int(pattern.match(dir_names[menu.value]).groups()[1])

# Plot data

In [ ]:
# in test_data
rows=int(len(paths) / 3) + (len(paths) % 3 > 0) # how many rows
size=np.int(np.sqrt(len(data[0][:,2])))
# plot
fig = plt.figure(figsize=(23, 6.7*rows))
for i,image in enumerate(data):
    ax=plt.subplot(rows,3,i+1)
#     #########################
#     ax=plt.subplot(3,2,i+1)
#     #########################
    im=ax.imshow(image[:,2].reshape((size,size)),cmap='jet', interpolation='nearest')
    # xticks
    ax.set_xticklabels(labels=image[:,0][:size][::4].astype(int))
    ax.set_xticks(range(size)[::4])
    ax.set_xlabel("x-pos in mm",fontsize=15)
    # yticks
    ax.set_yticklabels(labels=image[:,1][::21][::4].astype(int))
    ax.set_yticks(range(size)[::4])
    ax.set_ylabel("y-pos in mm",fontsize=15)
    ax.set_title(paths[i].name,fontsize=20)
    #adjust colorbar to plot
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    cbar=plt.colorbar(im, cax=cax)
    cbar.ax.set_ylabel('intensity',fontsize=15)
# print current directory
print("current directory:\n{}".format(dir_names[menu.value]))

# Export

plt.savefig("Images/beam_maps/beam_maps_{}mm.png".format(dir_dist), bbox_inches='tight')

# Project find beam profile

In [ ]:
def gaussian_profile(coords,A=1,b=0,x_0=0,y_0=0,σ_x=1,σ_y=1,θ=0,**kwargs):
    if ('popt' in kwargs):
        A,b,x_0,y_0,σ_x,σ_y,θ=popt[0],popt[1],popt[2],popt[3],popt[4],popt[5],popt[6]
    a=np.cos(θ)**2/(2*σ_x**2)+np.sin(θ)**2/(2*σ_y**2)
    c=np.sin(2*θ)*(-1/(4*σ_x**2)+1/(4*σ_y**2))
    d=np.sin(θ)**2/(2*σ_x**2)+np.cos(θ)**2/(2*σ_y**2)
    return A*np.exp(-(a*(coords[0]-x_0)**2+2*c*(coords[0]-x_0)*(coords[1]-y_0)+d*(coords[1]-y_0)**2))+b
# Was das Modell nicht berücksichtigt:
    # Auftreffswinkel

In [ ]:
# caclulate fits
popt_data=[]
pcov_data=[]
for image in data:
    max_pos=np.where(image[:,2]==np.amax(image[:,2]))
    initial_guess = (np.amax(image),0,np.int(image[max_pos,0]),np.int(image[max_pos,1]),20,20,0)
    popt, pcov = opt.curve_fit(gaussian_profile, (image[:,0], image[:,1]), image[:,2], p0=initial_guess)
    pcov=np.sqrt(np.diag(pcov))
    popt_data.append(popt)
    pcov_data.append(pcov)

In [ ]:
# plot the fits and functions in 3D
rows=int(len(popt_data) / 3) + (len(popt_data) % 3 > 0) # how many rows
fig = plt.figure(figsize=(24,rows*7))

for i,(popt,image) in enumerate(zip(popt_data,data)):
    X=np.linspace(np.amin(image[:,0]),np.amax(image[:,0]),50)
    Y=np.linspace(np.amin(image[:,1]),np.amax(image[:,1]),50)
    X_grid,Y_grid=np.meshgrid(X,Y)
    #plot
    ax = fig.add_subplot(rows,3,i+1, projection='3d')
    ax.set_xlabel('X-pos in mm', fontsize=15)
    ax.set_ylabel('Y-pos in mm', fontsize=15)
    ax.set_zlabel('Intensität', fontsize=15)
    Z=[gaussian_profile((x,y),popt=popt) for x in X for y in Y]
    ax.plot_trisurf(image[:,0], image[:,1], image[:,2], cmap='Blues',alpha=0.3)
    ax.plot_trisurf(X_grid.ravel(), Y_grid.ravel(), Z, cmap='Reds',alpha=0.4 )
    ax.set_title(paths[i].name, fontsize=20)
# legend
    blue_proxy = plt.Rectangle((0, 0), 1, 1, fc="Blue",alpha=0.3)
    red_proxy = plt.Rectangle((0, 0), 1, 1, fc="red",alpha=0.4)
    ax.view_init(65,-45) #angle of view
fig.legend([blue_proxy,red_proxy],['data','gauss-fit'],loc=1,prop={"size":40})
# print current directory
print("current directory:\n{}".format(dir_names[menu.value]))

# export

plt.savefig("Images/Gauss_fits_3D/Gauss_fits_3D_{}mm.png".format(dir_dist), bbox_inches='tight')

In [ ]:
# plot the fits and functions
rows=int(len(popt_data) / 3) + (len(popt_data) % 3 > 0) # how many rows
fig = plt.figure(figsize=(20,rows*5))

for i,(popt,image) in enumerate(zip(popt_data,data)):
    #max positions
    max_pos=np.where(image[:,2]==np.amax(image[:,2]))[0][0]
    x_max=np.int(image[:,0][max_pos])
    y_max=np.int(image[:,1][max_pos])
    #plot
    ax = fig.add_subplot(rows,3,i+1)
    ax.set_xlabel('x- or y-pos in mm', fontsize=25)
    ax.set_ylabel('Intensity', fontsize=25)
    ax.set_title(paths[i].name, fontsize=20)
    # x-plot
    X_vals=image[:,0][np.where(image[:,1]==y_max)]
    X_ints=image[:,2][np.where(image[:,1]==y_max)]
    X_fit=[gaussian_profile((x,y_max),popt=popt) for x in X_vals]
    plt.bar(X_vals,X_ints,width=4,color="blue",edgecolor="black",alpha=0.3,label="x-data")
    plt.plot(X_vals,X_fit,color="blue",label="x-fit")
    # y_plot
    Y_vals=image[:,1][np.where(image[:,0]==x_max)]
    Y_ints=image[:,2][np.where(image[:,0]==x_max)]
    Y_fit=[gaussian_profile((x_max,y),popt=popt) for y in Y_vals]
    plt.bar(Y_vals,Y_ints,width=4,color="red",edgecolor="black",alpha=0.3,label="y-data")
    plt.plot(Y_vals,Y_fit,color="red",label="y-fit")
    plt.legend(prop={"size":10})
plt.tight_layout()
# print current directory
print("current directory:\n{}".format(dir_names[menu.value]))

# export

plt.savefig("Images/Gauss_fits_xy/Gauss_fits_xy_{}mm.png".format(dir_dist), bbox_inches='tight')

In [ ]:
popt_array=np.array(popt_data)
pcov_array=np.array(pcov_data)
total_array=np.zeros(shape=(popt_array.shape[0],popt_array.shape[1]*2))
for i in range(0,total_array.shape[1]):
    if i%2==0:
        total_array[:,i]=popt_array[:,np.int(i/2)]
    else:
        total_array[:,i]=pcov_array[:,np.int((i-1)/2)]

In [ ]:
# create Pandas DataFrame
popt_array=np.array(popt_data)
pcov_array=np.array(pcov_data)
total_array=np.zeros(shape=(popt_array.shape[0],popt_array.shape[1]*2))
for i in range(0,total_array.shape[1]):
    if i%2==0:
        total_array[:,i]=popt_array[:,np.int(i/2)]
    else:
        total_array[:,i]=pcov_array[:,np.int((i-1)/2)]
#DataFrame initialization
fit_results=pd.DataFrame(total_array,index=list(map(lambda x: x.name, paths)),
                     columns=["A","ΔA","b","Δb","x_0","Δx_0","y_0","Δy_0","σ_x","Δσ_x","σ_y","Δσ_y","θ","Δθ"]).round(decimals=2)
fit_results["θ"],fit_results["Δθ"]=(fit_results["θ"]%(2*np.pi)).round(decimals=2),(fit_results["Δθ"]%(2*np.pi)).round(decimals=2)

In [ ]:
# In the following we assume that σ_x=σ_y und therefore take the average:
total_results=pd.DataFrame([(fit_results["σ_x"]+fit_results["σ_y"])/2,
                            np.sqrt((fit_results["Δσ_x"]**2+fit_results["Δσ_y"]**2))/2],
                           index=["σ","Δσ"]).T.round(decimals=2)
total_results["ω_z"]=2*total_results["σ"]
total_results["Δω_z"]=2*total_results["Δσ"]
total_results

# Comparison of theory and experiment

## Standard Calculation

In [ ]:
# find all frequencies in our data
path_names=list(map(lambda x: x.name, paths))
pattern=re.compile(r"(\D+)(\d+)")
freq_list=np.array([pattern.match(x).groups()[1] for x in path_names]).astype(int)
frequencies=np.unique(freq_list) # in GHz
# error bars
freq_range=np.linspace(min(frequencies),max(frequencies),100)
# # # # # # # #
d_in_err=10
d_mirr_device=15
X_err=np.sqrt((d_in_err)**2+(d_mirr_device)**2)
foc_err=0
freq_err=1.25
err_precision=1
# # # # # # # # 
X_bar=np.arange(dir_dist-X_err,dir_dist+X_err+err_precision,err_precision)
d_in_bar=np.arange(d_in-d_in_err,d_in+d_in_err+err_precision,err_precision)
foc_bar=np.arange(foc-foc_err,foc+foc_err+err_precision,err_precision)
error_area=np.zeros((len(X_bar)*len(d_in_bar)*len(np.arange(freq-freq_err,freq+freq_err+err_precision,err_precision))*len(foc_bar),len(freq_range)))
# # # # # # # #
for i,freq in tqdm(enumerate(freq_range),total=len(freq_range),desc="progress"):
    j=0
    freq_bar=np.arange(freq-freq_err,freq+freq_err+err_precision,err_precision)
    for err_X in X_bar:
        for err_d_in in d_in_bar:
            for err_foc in foc_bar:
                for err_freq in freq_bar:
                    error_area[j,i]=create_plot(X=err_X, d_in=err_d_in,ω_0_in=ω_0_in_list[min(ω_0_in_list.keys(), key=lambda x:abs(x-err_freq))],foc=err_foc,λ=c/(err_freq*1E9))
                    j+=1
#plot experimental data
ax=plt.subplot(111)
ax.errorbar(freq_list.astype(int),np.array(total_results["ω_z"]),linestyle="",
             yerr=np.array(total_results["Δω_z"]), color="red",marker=".",label="Experiment")
# plot theory data without errors
clean_theory=[create_plot(X=dir_dist, d_in=d_in,ω_0_in=ω_0_in_list[min(ω_0_in_list.keys(), key=lambda x:abs(x-freq))],foc=foc,λ=c/(freq*1E9)) for freq in freq_range]
ax.plot(freq_range,clean_theory,color="blue",linewidth=1, label="Theory without errors")
#plot error
ax.plot(freq_range,error_area.max(axis=0), color="blue",linewidth=0.5, label="Theory")
ax.plot(freq_range,error_area.min(axis=0), color="blue",linewidth=0.5)
ax.fill_between(freq_range, error_area.max(axis=0), error_area.min(axis=0), facecolor='blue', interpolate=True,alpha=0.1)
# legend
handles, labels = ax.get_legend_handles_labels()
ax.legend([handles[2][0],handles[1],handles[0]],labels[::-1],loc='upper right', numpoints=1)
#ax.set_title("Comparison between Theory and Experiment",fontsize=15)
ax.set_xlabel("Frequency in GHz",fontsize=15)
ax.set_ylabel("Beam Width ω_z in mm",fontsize=15)
ax.grid(True) # activate grid in plot
# print current directory
print("current directory:\n{}".format(dir_names[menu.value]))

# export

plt.savefig("Images/comp_standard/comp_standard_{}mm.png".format(dir_dist), bbox_inches='tight')

## Heterodyne Calculation

### Without errors, but 100 frequencies per band

In [ ]:
# find all frequencies in our data
path_names=list(map(lambda x: x.name, paths))
pattern=re.compile(r"(\D+)(\d+)")
freq_list=np.array([pattern.match(x).groups()[1] for x in path_names]).astype(int)

In [ ]:
# find all frequencies in our data
path_names=list(map(lambda x: x.name, paths))
pattern=re.compile(r"(\D+)(\d+)")
freq_list=np.array([pattern.match(x).groups()[1] for x in path_names]).astype(int)
# rest
band=100 # elements per band (discrete approximation for continuous value)

# Caculation
X_a=[list(np.linspace(x-6.35-1.25,x-6.35+1.25,band))+list(np.linspace(x+6.35-1.25,x+6.35+1.25,band)) for x in freq_list]
X_b=[[1/len(X_a[0])]*len(X_a[0])]*len(X_a)
Y=[I_comb(dir_dist, f_list=X_a[i], A_0_list=X_b[i],d_in=d_in,foc=foc,c=c,ω_0_in_list=ω_0_in_list) for i in tqdm(range(len(X_a)))]
ax= plt.subplot(1,1,1)
ax.scatter(freq_list,Y,color="blue",s=10,label="intensity by two frequenciey bands (1000 points per band)")
ax.errorbar(freq_list.astype(int),np.array(total_results["ω_z"]),linestyle="",
             yerr=np.array(total_results["Δω_z"]), color="red",marker=".",label="Experiment")
ax.set_xlabel("frequency ν in GHz",fontsize=14)
ax.set_ylabel("beam radius w(z) in mm",fontsize=14)
ax.grid(True) # activate grid in plot
plt.legend(bbox_to_anchor=(0.0, 1.17), loc='upper left', borderaxespad=0.)


### With errors, but less frequencies in band

In [ ]:
# find all frequencies in our data
path_names=list(map(lambda x: x.name, paths))
pattern=re.compile(r"(\D+)(\d+)")
freq_list=np.array([pattern.match(x).groups()[1] for x in path_names]).astype(int)
frequencies=np.unique(freq_list) # in GHz
# error bars
freq_range=np.linspace(min(frequencies),max(frequencies),100)
# # # # # # # #
d_in_err=10
d_mirr_device=15
X_err=np.sqrt((d_in_err)**2+(d_mirr_device)**2)
foc_err=0
freq_err=0
err_precision=5
band=20
# # # # # # # # 
X_bar=np.arange(dir_dist-X_err,dir_dist+X_err+err_precision,err_precision)
d_in_bar=np.arange(d_in-d_in_err,d_in+d_in_err+err_precision,err_precision)
foc_bar=np.arange(foc-foc_err,foc+foc_err+err_precision,err_precision)
error_area=np.zeros((len(X_bar)*len(d_in_bar)*len(np.arange(freq-freq_err,freq+freq_err+err_precision,err_precision))*len(foc_bar),len(freq_range)))
# # # # # # # #
for i,freq in tqdm(enumerate(freq_range),total=len(freq_range),desc="total progress"):
    j=0
    freq_bar=np.arange(freq-freq_err,freq+freq_err+err_precision,err_precision)
    for err_X in tqdm(X_bar,leave=False):
        for err_d_in in d_in_bar:
            for err_foc in foc_bar:
                for err_freq in freq_bar:
                    # Caculation
                    X_a=list(np.linspace(err_freq-6.35-1.25,err_freq-6.35+1.25,band)) + list(np.linspace(err_freq+6.35-1.25,err_freq+6.35+1.25,band))
                    X_b=[1/len(X_a)]*len(X_a)
                    error_area[j,i]=I_comb(err_X, f_list=X_a, A_0_list=X_b,d_in=err_d_in,foc=err_foc,c=c,ω_0_in_list=ω_0_in_list)
                    j+=1
#plot experimental data
ax=plt.subplot(111)
ax.errorbar(freq_list.astype(int),np.array(total_results["ω_z"]),linestyle="",
             yerr=np.array(total_results["Δω_z"]), color="red",marker=".",label="Experiment")
# plot theory data without errors
X_a=[list(np.linspace(x-6.35-1.25,x-6.35+1.25,band))+list(np.linspace(x+6.35-1.25,x+6.35+1.25,band)) for x in freq_list]
X_b=[[1/len(X_a[0])]*len(X_a[0])]*len(X_a)
Y=[I_comb(dir_dist, f_list=X_a[i], A_0_list=X_b[i],d_in=d_in,foc=foc,c=c,ω_0_in=ω_0_in) for i in tqdm(range(len(X_a)))]
ax.scatter(freq_list,Y,color="blue", label="Theory without errors")
#plot error
ax.plot(freq_range,error_area.max(axis=0), color="blue",linewidth=0.5, label="Theory")
ax.plot(freq_range,error_area.min(axis=0), color="blue",linewidth=0.5)
ax.fill_between(freq_range, error_area.max(axis=0), error_area.min(axis=0), facecolor='blue', interpolate=True,alpha=0.1)
# legend
handles, labels = ax.get_legend_handles_labels()
ax.legend([handles[2][0],handles[1],handles[0]],labels[::-1],loc='upper right', numpoints=1)
#ax.set_title("Comparison between Theory and Experiment",fontsize=15)
ax.set_xlabel("Frequency in GHz",fontsize=15)
ax.set_ylabel("Beam Width ω_z in mm",fontsize=15)
ax.grid(True)
# print current directory
print("current directory:\n{}".format(dir_names[menu.value]))

# export

plt.savefig("Images/comp_heterodyne/comp_heterodyne_{}mm.png".format(dir_dist), bbox_inches='tight')